In [4]:
import pandas as pd
import numpy as np
import pygrib as gb
import os

In [6]:
df = pd.read_csv('/Volumes/Seagate/ano_decade_wind.csv', parse_dates=['Data_Date', 'Validity_Date'], na_values='--')

#### Calculate wind vectors

Obtain a vector projection of the wind vector (in cartesian coordinates) on a unit vector pointing onshore.

In [7]:
def get_cartesian_projection(row, unit_vector):
    wdir = row['Wind_Direction']
    wdspd = row['Wind_Speed']
    wdir -= 270
    if wdir < 0:
        wdir += 360
    x = wdspd*np.cos(np.deg2rad(wdir))
    y = wdspd*np.sin(np.deg2rad(wdir))
    projection = np.dot([x,y], unit_vector)
    return projection

In [8]:
onshore_unit_vector = np.array([1*np.cos(np.deg2rad(45)), 1*np.sin(np.deg2rad(45))])

In [9]:
df['Projection'] = df.apply(lambda row: get_cartesian_projection(row, onshore_unit_vector), axis=1)

In [10]:
df['dDate'] = df['Validity_Date'] - df['Data_Date']

In [11]:
df = df.loc[df.groupby('Validity_Date')['dDate'].idxmin()] #Get forecast closest to the validity date

In [12]:
wdvectors = pd.read_csv('/Volumes/Seagate/ano_decade_wind_min.csv', parse_dates=['Validity_Date'])
strandings = pd.read_csv('/Volumes/Seagate/2007-2018_es_strandings.csv', parse_dates=['Strand_Date'])

In [13]:
def get_ano_nuevo_decade_window(df, time_lag):
    averages = (df[['Validity_Date','Projection']].rolling('36h', on='Validity_Date')
                .mean())
    averages['Validity_Date'] = averages['Validity_Date'] + pd.Timedelta(str(time_lag) + 'h')
    return averages

In [14]:
averages = get_ano_nuevo_decade_window(wdvectors, 24)

In [25]:
strandings = pd.merge(left=strandings, right=averages, left_on='Strand_Date', right_on='Validity_Date')
strandings = strandings[(strandings['Strand_Date'].dt.dayofyear < 45) & (strandings['Age_Class'].isin(['Pup', 'Weaner']))]